In [34]:
# Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import os
from datetime import timedelta, datetime, tzinfo
from zipfile import ZipFile
import collections


In [35]:
## Marker Cluster Map ##

In [36]:
# Storing CSV data from 2013 to 2018 into DataFrame
csv_file = os.path.join('static','data/gun-violence-data_01-2013_03-2018.csv')
gun_violence_data_df = pd.read_csv(csv_file)
gun_violence_data_df.head()


,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,...,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,...,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [37]:
# Creating new dataframe
gv_data_df = gun_violence_data_df[['date','state','city_or_county','n_killed','n_injured','latitude','longitude','incident_url','source_url']].copy()
gv_data_df.head()


,date,state,city_or_county,n_killed,n_injured,latitude,longitude,incident_url,source_url
0,2013-01-01,Pennsylvania,Mckeesport,0,4,40.3467,-79.8559,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...
1,2013-01-01,California,Hawthorne,1,3,33.9090,-118.3330,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...
2,2013-01-01,Ohio,Lorain,1,3,41.4455,-82.1377,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...
3,2013-01-05,Colorado,Aurora,4,0,39.6518,-104.8020,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...
4,2013-01-07,North Carolina,Greensboro,2,2,36.1140,-79.9569,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...


In [38]:
## Heat Map data saving to csv file ##
#gv_data_df.to_csv('static/data/Heatmap_2013_2018_data.csv')

In [39]:
# Converting column type to string
gv_data_df['date'] = gv_data_df['date'].astype(str)
gv_data_df['date'].dtype

dtype('O')

In [40]:
# Creating new dataframe from old one
df = gv_data_df

In [41]:
# Filtering out rows between Jan 2018 and March 2018
gv_data_2018 = df[(df['date'] >= '2018-01-01') & (df['date'] <= '2018-03-31')]

# Writing dataframe into csv file
#gv_data_2018.to_csv('static/data/Marker_Cluster_Data_2018.csv')

In [42]:
# 2018 data
gv_data_2018

,date,state,city_or_county,n_killed,n_injured,latitude,longitude,incident_url,source_url
225875,2018-01-01,New Jersey,Trenton,0,0,40.2082,-74.7455,http://www.gunviolencearchive.org/incident/101...,http://www.trentonian.com/general-news/2018010...
225876,2018-01-01,Massachusetts,Boston,0,0,42.3379,-71.0547,http://www.gunviolencearchive.org/incident/101...,https://twitter.com/southiescanner/status/9479...
225877,2018-01-01,Massachusetts,Boston,0,0,42.3324,-71.0517,http://www.gunviolencearchive.org/incident/101...,https://twitter.com/southiescanner/status/9479...
225878,2018-01-01,California,Oakland,1,0,37.7698,-122.2040,http://www.gunviolencearchive.org/incident/101...,https://www.eastbaytimes.com/2018/01/03/man-ch...
225879,2018-01-01,New York,Buffalo,0,3,42.9093,-78.8491,http://www.gunviolencearchive.org/incident/102...,http://buffalonews.com/2018/01/07/car-riddled-...
225880,2018-01-01,California,Auburn,0,0,38.9473,-121.1020,http://www.gunviolencearchive.org/incident/102...,http://www.auburnjournal.com/article/1/15/18/s...
225881,2018-01-01,Arkansas,Little Rock,0,1,34.7204,-92.4523,http://www.gunviolencearchive.org/incident/102...,http://www.arkansasonline.com/news/2018/jan/05...
225882,2018-01-01,California,Garberville,1,0,40.1008,-123.6870,http://www.gunviolencearchive.org/incident/101...,https://lostcoastoutpost.com/2018/jan/3/sherif...
225883,2018-01-01,Michigan,Kalamazoo,0,0,42.3034,-85.5888,http://www.gunviolencearchive.org/incident/101...,http://www.mlive.com/news/kalamazoo/index.ssf/...
225884,2018-01-01,Ohio,Lorain,1,0,41.4494,-82.1760,http://www.gunviolencearchive.org/incident/101...,http://www.chroniclet.com/cops-and-courts/2018...


In [43]:
# Connection to database
connection_string = "postgres:Science2019@localhost:5432/GUN_VIOLENCE_PROJECT"
engine = create_engine(f'postgresql://{connection_string}')
# Check for tables
engine.table_names()


['killings_injuries_2018',
 'gunshootingstype',
 'markercluster',
 'gun_year',
 'guns_year']

In [44]:
# Use pandas to load csv converted DataFrame into database table
gv_data_2018.to_sql(name='markercluster', con=engine, if_exists='replace', index=False)


In [47]:
# Confirm data has been added by querying the table
data = pd.read_sql_query('select * from markercluster', con=engine)
data.head()

,date,state,city_or_county,n_killed,n_injured,latitude,longitude,incident_url,source_url
0,2018-01-01,New Jersey,Trenton,0,0,40.2082,-74.7455,http://www.gunviolencearchive.org/incident/101...,http://www.trentonian.com/general-news/2018010...
1,2018-01-01,Massachusetts,Boston,0,0,42.3379,-71.0547,http://www.gunviolencearchive.org/incident/101...,https://twitter.com/southiescanner/status/9479...
2,2018-01-01,Massachusetts,Boston,0,0,42.3324,-71.0517,http://www.gunviolencearchive.org/incident/101...,https://twitter.com/southiescanner/status/9479...
3,2018-01-01,California,Oakland,1,0,37.7698,-122.2040,http://www.gunviolencearchive.org/incident/101...,https://www.eastbaytimes.com/2018/01/03/man-ch...
4,2018-01-01,New York,Buffalo,0,3,42.9093,-78.8491,http://www.gunviolencearchive.org/incident/102...,http://buffalonews.com/2018/01/07/car-riddled-...


In [14]:
## Choropleth map ##

In [15]:
## Reading Mass Shootings 2019 Data

In [17]:
# Store CSV into DataFrame (Movie)
csv_file_2019 = os.path.join('static/data/Mass_Shootings_2019.csv')
mass_shootings_2019_df = pd.read_csv(csv_file_2019)
mass_shootings_2019_df.head()


,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations
0,"December 15, 2019",Georgia,Columbus,600 block of 32nd St,1,4,NaN
1,"December 12, 2019",Missouri,Saint Louis,9900 block of Lewis and Clark Blvd,1,3,NaN
2,"December 10, 2019",New Jersey,Jersey City,223 Martin Luther King Dr,6,3,NaN
3,"December 8, 2019",Texas,Desoto,200 block of W Wintergreen Rd,2,3,NaN
4,"December 8, 2019",Louisiana,New Orleans,3801 Washington Ave,1,4,NaN


In [19]:
# Renaming column names
mass_shootings_2019_df = mass_shootings_2019_df.rename(columns={'# Killed':'n_killed', '# Injured':'n_injured'})

In [20]:
# Grouping dataframe by state
mass_shootings_2019_df_group = mass_shootings_2019_df.groupby('State')

In [21]:
# Calculation total injuries and killings in 2019 mass shootings
mass_shootings_2019_df_sum = mass_shootings_2019_df_group['n_killed','n_injured'].sum()

In [22]:
# Calculationg total number of mass shooting incidents in 2019 - used these countings in geojson file
mass_shootings_2019_df_count = mass_shootings_2019_df_group['Incident Date'].count()
mass_shootings_2019_df_count

State
Alabama                 11
Arizona                  6
Arkansas                 5
California              45
Colorado                 4
Connecticut              3
Delaware                 2
District of Columbia     7
Florida                 14
Georgia                 18
Illinois                38
Indiana                 10
Iowa                     2
Kansas                   4
Kentucky                 5
Louisiana               24
Maryland                20
Massachusetts            5
Michigan                 9
Minnesota                1
Mississippi              5
Missouri                18
Montana                  1
Nebraska                 1
Nevada                   3
New Jersey              11
New Mexico               5
New York                 8
North Carolina          10
Ohio                    14
Oklahoma                 4
Oregon                   1
Pennsylvania            19
South Carolina          10
Tennessee                7
Texas                   29
Virginia              

In [23]:
## Bar Chart ##

In [24]:
gun_df = gun_violence_data_df

In [25]:
# Calculating injuries, killings and gun counts for years 2014,15,16 and 17

date_df = gun_df.set_index("date")
##
date_df.sort_values(by=['date'], inplace=True, ascending=False)
data2017_df = date_df.loc['2017-12-31':'2017-01-01']
data2016_df = date_df.loc['2016-12-31':'2016-01-01']
data2015_df = date_df.loc['2015-12-31':'2015-01-01']
data2014_df = date_df.loc['2014-12-31':'2014-01-01']
##
trend_list = [{"year":"2017","injured": data2017_df["n_injured"].sum(), "killed":data2017_df["n_killed"].sum(),
              "guns_count":data2017_df["n_guns_involved"].sum() },
             {"year":"2016","injured": data2016_df["n_injured"].sum(), "killed":data2016_df["n_killed"].sum(),
              "guns_count":data2016_df["n_guns_involved"].sum() },
             {"year":"2015","injured": data2015_df["n_injured"].sum(), "killed":data2015_df["n_killed"].sum(),
              "guns_count":data2015_df["n_guns_involved"].sum() },
             {"year":"2014","injured": data2014_df["n_injured"].sum(), "killed":data2014_df["n_killed"].sum(),
              "guns_count":data2014_df["n_guns_involved"].sum() }]


trend_list
##
years_trend = pd.DataFrame.from_dict(trend_list)
years_trend

,guns_count,injured,killed,year
0,76189.0,30703,15511,2017
1,53821.0,30580,15066,2016
2,36849.0,26967,13484,2015
3,7927.0,23002,12557,2014


In [26]:
## Line Chart ##

In [29]:
# Calculating Gun related killings and injuries each month in year 2014

data14to18_df = date_df.loc['2018-03-31':'2014-01-01']
resetindex14_df = data14to18_df.reset_index()
new_df = resetindex14_df[["date","n_killed","n_injured", "n_guns_involved"]]
permonth1418 = pd.DatetimeIndex(new_df.date).to_period("M") 
gun14_monthly =new_df.groupby(permonth1418).sum()
gun14_monthly.reset_index(inplace=True)
gun14_monthly.head(12)


,date,n_killed,n_injured,n_guns_involved
0,2014-01,1070,1875,406.0
1,2014-02,845,1223,317.0
2,2014-03,1007,1551,406.0
3,2014-04,979,1782,586.0
4,2014-05,1099,2100,429.0
5,2014-06,1127,2069,486.0
6,2014-07,1162,2423,494.0
7,2014-08,1140,2298,492.0
8,2014-09,1013,2059,658.0
9,2014-10,1012,1884,705.0


In [32]:
# Inserting Bar chart dataframe to SQL
years_trend.to_sql(name='guns_year', con=engine, if_exists='replace', index=False)
# Confirm data has been added by querying the table
data_bar = pd.read_sql_query('select * from guns_year', con=engine).head()
data_bar

,guns_count,injured,killed,year
0,76189.0,30703,15511,2017
1,53821.0,30580,15066,2016
2,36849.0,26967,13484,2015
3,7927.0,23002,12557,2014


In [33]:
#Creating CSV's of Bar and Line dataframes
years_trend.to_csv('static/data/year_trend_gun.csv')
gun14_monthly.to_csv("static/data/gun2014onwrd.csv")
